In [99]:
import pandas as pd
import numpy as np
import re
import string
import warnings
warnings.filterwarnings("ignore")


In [100]:
abbr_name = pd.read_csv('./data/abbr-name.csv')
abbr_name['abbr'] = abbr_name['abbr'].str.lower()
state = abbr_name['abbr'].to_list()

# Cost of living data 2012-2021

In [101]:
cost_of_living_12 = pd.read_csv('./data/Cost_living_2012.csv')
cost_of_living_13 = pd.read_csv('./data/Cost_living_2013.csv')
cost_of_living_14 = pd.read_csv('./data/Cost_living_2014.csv')
cost_of_living_15 = pd.read_csv('./data/Cost_living_2015.csv')
cost_of_living_16 = pd.read_csv('./data/Cost_living_2016.csv')
cost_of_living_17 = pd.read_csv('./data/Cost_living_2017.csv')
cost_of_living_18 = pd.read_csv('./data/Cost_living_2018.csv')
cost_of_living_19 = pd.read_csv('./data/Cost_living_2019.csv')
cost_of_living_20 = pd.read_csv('./data/Cost_living_2020.csv')
cost_of_living_21 = pd.read_csv('./data/Cost_living_2021.csv')

cost_living_12_21 = pd.concat([cost_of_living_12, cost_of_living_13, cost_of_living_14, cost_of_living_15, cost_of_living_16, 
                                cost_of_living_17, cost_of_living_18,cost_of_living_19, cost_of_living_20, cost_of_living_21])
cost_living_12_21['USstate'] = cost_living_12_21['USstate'].str.lower()
cost_living_12_21 = cost_living_12_21.drop("Unnamed: 0",axis=1)
cost_living_12_21

,Year,USstate,Cost of Living Index
0,2012,ma,106.80
1,2012,hi,103.46
2,2012,ca,102.52
3,2012,wa,101.04
4,2012,ny,100.00
...,...,...,...
34,2021,ok,64.61
35,2021,ut,63.89
36,2021,id,63.30
37,2021,ar,62.89


In [102]:
# cost_living_12_21['year'] = cost_living_12_21['Year']
cost_living_12_21.rename(columns={'Year': 'year'}, inplace=True)
cost_living_12_21 = cost_living_12_21.groupby(['year','USstate']).sum().reset_index()
cost_of_living_state = set(cost_living_12_21['USstate'].to_list())



year = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,2021]
month = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
col_all = pd.DataFrame(columns=['USstate', 'year', 'month', 'factor'])
for i in range(len(cost_of_living_state)):
    for j in year:
        for k in month:
            if k == 12:
                col_all = col_all.append({'USstate': state[i], 'year': j, 'month': k, 'factor': 1}, 
                                                   ignore_index=True)
            else:
                col_all = col_all.append({'USstate': state[i], 'year': j, 'month': k, 'factor': 0}, 
                                                   ignore_index=True)

In [148]:
cost_living_data = pd.merge(cost_living_12_21, col_all, on=['USstate', 'year'], how='right')
cost_living_data['Cost of Living Index'] = cost_living_data['Cost of Living Index'] * cost_living_data['factor']
cost_living_data = cost_living_data.drop(columns=['factor'])
cost_living_data['Cost of Living Index'] = cost_living_data['Cost of Living Index'].replace({0:np.nan})
cost_living_data['Cost of Living Index'] = cost_living_data['Cost of Living Index'].interpolate(method ='cubic', limit_direction ='backward')
cost_living_data['Cost of Living Index'] = cost_living_data['Cost of Living Index'].interpolate(method ='linear', limit_direction ='backward')
cost_living_data['year'] = pd.to_numeric(cost_living_data['year'])
cost_living_data['month'] = pd.to_numeric(cost_living_data['month'])
cost_living_data.to_csv('./data/cost of living/cost_living_data.csv')
cost_living_data

,year,USstate,Cost of Living Index,month
0,2012,al,71.180000,1
1,2012,al,71.180000,2
2,2012,al,71.180000,3
3,2012,al,71.180000,4
4,2012,al,71.180000,5
...,...,...,...,...
5275,2021,tx,68.324387,8
5276,2021,tx,68.391636,9
5277,2021,tx,68.459544,10
5278,2021,tx,68.526276,11


# 4. Merge all datasets

In [142]:
tweet_df = pd.read_csv('./data/tweet/tweets_df.csv',index_col=[0])
crime_df = pd.read_csv('./data/crime/crime.csv',index_col=[0])
merge_temp_data = pd.read_csv('./data/temperature/mergedtemperature.csv',index_col=[0])

In [143]:
merge_tweet_cost = pd.merge(tweet_df ,cost_living_data, left_on=['USstate', 'year','month'], right_on=['USstate', 'year','month'], how='left')
merge_tweet_cost = pd.merge(merge_tweet_cost,crime_df,left_on=['USstate', 'year','month'], right_on=['USstate', 'year','month'], how='left')
merge_tweet_cost.dropna()

,USstate,year,month,nTweets_x,nTweets_y,Cost of Living Index,population,actual_rape_total
0,al,2012,1,51744.0,3.0,71.180000,4834334.0,106.0
1,al,2012,2,51744.0,3.0,71.180000,4834334.0,70.0
2,al,2012,3,55089.0,2.0,71.180000,4834334.0,78.0
3,al,2012,4,38471.0,1.0,71.180000,4834334.0,86.0
4,al,2012,5,36891.0,1.0,71.180000,4834334.0,101.0
...,...,...,...,...,...,...,...,...
5263,tx,2020,8,711151.0,6.0,68.235824,29436098.0,1215.0
5264,tx,2020,9,669447.0,3.0,68.152224,29436098.0,1220.0
5265,tx,2020,10,708416.0,5.0,68.091275,29436098.0,1158.0
5266,tx,2020,11,709005.0,3.0,68.051145,29436098.0,1021.0


In [140]:
merge_new = pd.merge(merge_temp_data,cost_living_data,left_on=['USstate', 'year','month'], right_on=['USstate', 'year','month'], how='left')
merge_new['tweet_proportion'] = merge_new['tweet_proportion_x']
merge_new = merge_new.drop(columns=['tweet_proportion_x','tweet_proportion_y'])
merge_new.dropna()

,USstate,year,month,nTweets_x,nTweets_y,Number of establishments (ESTAB),abbr,full,population,actual_rape_total,Temperature,Cost of Living Index,tweet_proportion
0,al,2012,1,51744.0,3.0,367.000000,al,Alabama,4834334,106,50.1,71.180000,0.000058
1,al,2012,2,51744.0,3.0,367.000000,al,Alabama,4834334,70,52.1,71.180000,0.000058
2,al,2012,3,55089.0,2.0,367.000000,al,Alabama,4834334,78,64.6,71.180000,0.000036
3,al,2012,4,38471.0,1.0,367.000000,al,Alabama,4834334,86,65.0,71.180000,0.000026
4,al,2012,5,36891.0,1.0,367.000000,al,Alabama,4834334,101,73.4,71.180000,0.000027
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4747,tx,2020,8,711151.0,6.0,2235.626743,tx,Texas,29436098,1215,84.4,68.235824,0.000008
4748,tx,2020,9,669447.0,3.0,2219.773590,tx,Texas,29436098,1220,74.1,68.152224,0.000004
4749,tx,2020,10,708416.0,5.0,2183.055271,tx,Texas,29436098,1158,65.9,68.091275,0.000007
4750,tx,2020,11,709005.0,3.0,2122.216002,tx,Texas,29436098,1021,60.6,68.051145,0.000004


In [144]:
merge_new.to_csv('./data/merged_data.csv')